In [60]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.60.0
Summary: Used to build and run machine learning workflows upon the     Azure Machine Learning service.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: 
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages
Requires: azureml-core, azureml-dataset-runtime, azureml-pipeline, azureml-train-automl-client, azureml-train-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install azureml-sdk==1.48.0

ERROR: Could not find a version that satisfies the requirement azureml-sdk==1.48.0 (from versions: 0.1.57, 0.1.58, 0.1.59, 0.1.65, 0.1.68, 0.1.74, 0.1.80, 1.0rc83, 1.0rc85, 1.0.2, 1.0.6, 1.0.8, 1.0.10, 1.0.15, 1.0.15.1, 1.0.17, 1.0.18, 1.0.18.1, 1.0.21, 1.0.23, 1.0.30, 1.0.33, 1.0.39, 1.0.41, 1.0.43, 1.0.45, 1.0.48, 1.0.53, 1.0.55, 1.0.57, 1.0.60, 1.0.62, 1.0.65, 1.0.69, 1.0.72, 1.0.74, 1.0.76, 1.0.79, 1.0.81, 1.0.83, 1.0.85, 1.1.0rc0, 1.1.1rc0, 1.1.2rc0, 1.1.5, 1.1.5.1, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.7.0.post1, 1.8.0, 1.9.0, 1.10.0, 1.11.0, 1.12.0, 1.13.0, 1.14.0, 1.15.0, 1.16.0, 1.17.0, 1.18.0, 1.19.0, 1.20.0, 1.21.0, 1.22.0, 1.23.0, 1.24.0, 1.25.0, 1.49.0, 1.50.0, 1.51.0, 1.52.0, 1.53.0, 1.54.0, 1.55.0, 1.56.0, 1.57.0, 1.58.0, 1.59.0, 1.60.0)
ERROR: No matching distribution found for azureml-sdk==1.48.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-280049
Azure region: westeurope
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-280049


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_nodes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [40]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 80.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install 'azureml-sdk'


Note: you may need to restart the kernel to use updated packages.


In [62]:
#from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from collections.abc import Mapping
from azureml.widgets import RunDetails


hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_cb4aa370-a840-4964-bbdc-1830326118bd
Web View: https://ml.azure.com/runs/HD_cb4aa370-a840-4964-bbdc-1830326118bd?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-280049/workspaces/quick-starts-ws-280049&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-05-06T13:37:06.2543486Z][GENERATOR][WARNING]Space size : 12 is less than max total jobs : 100, only 12 jobs will be generated 
[2025-05-06T13:37:07.7238803Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-05-06T13:37:07.9777671Z][SCHEDULER][INFO]Scheduling job, id='HD_cb4aa370-a840-4964-bbdc-1830326118bd_0' 
[2025-05-06T13:37:08.1103198Z][SCHEDULER][INFO]Scheduling job, id='HD_cb4aa370-a840-4964-bbdc-1830326118bd_2' 
[2025-05-06T13:37:08.1091918Z][SCHEDULER][INFO]Scheduling job, id='HD_cb4aa370-a840-4964-bbdc-1830326118bd_1' 
[2025-05-06T13:37:08.1146537Z][SCHEDULER][INFO]Scheduling job, id='HD_cb4aa370-a840-4964-bbdc-1830326118bd_3' 
[2025-05

{'runId': 'HD_cb4aa370-a840-4964-bbdc-1830326118bd',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2025-05-06T13:37:05.769912Z',
 'endTimeUtc': '2025-05-06T13:56:48.069505Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '53767361-4690-41ea-82a8-18d2e57cc893',
  'user_agent': 'python/3.10.16 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_cb4aa370-a840-4964-bbdc-1830326118bd_4',
  'score': '0.9147851420247632',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_cb4aa370-a840-4964-bbdc-1830326118bd_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train'

In [17]:
#from collections.abc import Mapping
#from azureml.widgets import RunDetails

#RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
#import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_cb4aa370-a840-4964-bbdc-1830326118bd_4,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-280049', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-280049'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

In [12]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.5', '--max_iter', '100']


In [13]:
print(best_run.get_metrics())

{'Regularization Strength:': 0.5, 'Max iterations:': 100, 'Accuracy': 0.9147851420247632}


In [63]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [64]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [65]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [78]:
pip install 'azureml-sdk[azureml-train-automl-run]'

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Submit your automl run
from azureml.core.experiment import Experiment

experiment = Experiment(ws, 'project1automl')
automl_run = experiment.submit(config=automl_config, show_out=True)

2025-05-06 12:55:25.526266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746536127.039385    4770 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746536127.472044    4770 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746536131.429965    4770 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746536131.430011    4770 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746536131.430014    4770 computation_placer.cc:177] computation placer alr

Experiment,Id,Type,Status,Details Page,Docs Page
project1automl,AutoML_98489613-e9bc-4a51-92b0-de8ec7830971,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025/05/06 13:01:02 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025-05-06:13:27:49,300 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-06:13:27:51,2 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-06:13:27:51,6 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-06:13:27:52,374 WARNING  [_docstring_wrapper.py:27] Class StackEnsembleClassifier: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2025-05-06:13:2

In [79]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment
from azureml.train.automl.run import AutoMLRun

best_run, fitted_model = automl_run.get_output()

NameError: name 'automl_run' is not defined

In [67]:
# Retrieve and save your best automl model.
from azureml.core.experiment import Experiment

#best_automl_run, fitted_automl_model = run.get_output()
best_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_run)

Run(Experiment: udacity-project,
Id: HD_cb4aa370-a840-4964-bbdc-1830326118bd_4,
Type: azureml.scriptrun,
Status: Completed)


In [68]:
print(best_run.get_details())

{'runId': 'HD_cb4aa370-a840-4964-bbdc-1830326118bd_4', 'target': 'Udacity', 'status': 'Completed', 'startTimeUtc': '2025-05-06T13:52:50.996192Z', 'endTimeUtc': '2025-05-06T13:53:27.268418Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'Udacity', 'ContentSnapshotId': '53767361-4690-41ea-82a8-18d2e57cc893', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train_wv.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.5', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'Udacity', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name':

In [22]:
print(best_run.get_metrics())

{'recall_score_macro': 0.7602699935938156, 'matthews_correlation': 0.5568936062506467, 'recall_score_micro': 0.9168437025796662, 'accuracy': 0.9168437025796662, 'f1_score_weighted': 0.9140498824682612, 'f1_score_micro': 0.9168437025796662, 'average_precision_score_micro': 0.9814475606864068, 'precision_score_weighted': 0.9122750035222786, 'average_precision_score_weighted': 0.9551084559706451, 'balanced_accuracy': 0.7602699935938156, 'log_loss': 0.28055622351229703, 'AUC_macro': 0.9473833240476681, 'f1_score_macro': 0.7771626300791125, 'average_precision_score_macro': 0.8233896333647799, 'norm_macro_recall': 0.5205399871876313, 'AUC_weighted': 0.9473833240476682, 'precision_score_macro': 0.7980464830698052, 'recall_score_weighted': 0.9168437025796662, 'precision_score_micro': 0.9168437025796662, 'AUC_micro': 0.9806547042122495, 'weighted_accuracy': 0.9557363567548203, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_98489613-e9bc-4a51-92b0-de8ec7830971_29/confusion_matri

In [ ]:
#cluster.delete()